# Command Registry

In [ ]:
from ipylab import JupyterFrontEnd

app = JupyterFrontEnd()

In [ ]:
app.version

## List all commands

In [ ]:
app.commands.list_commands()

## Create a new console

In [ ]:
app.commands.execute('console:create', {
    'insertMode': 'split-right',
    'kernelPreference': {
        'shutdownOnClose': True,
    }
})

## Change the theme

In [ ]:
app.commands.execute('apputils:change-theme', { 'theme': 'JupyterLab Dark' })

## Create a new terminal

In [ ]:
app.commands.execute('terminal:create-new')

## Ready event

Some functionalities might require the `JupyterFrontEnd` widget to be ready on the frontend first.

This is for example the case when listing all the available commands, or retrieving the version with `app.version`.

The `on_ready` method can be used to register a callback that will be fired when the frontend is ready.

In [ ]:
from ipylab import JupyterFrontEnd
from ipywidgets import Output

app = JupyterFrontEnd()
out = Output()

def init():
    # show the first 5 commands
    cmds = app.commands.list_commands()[:5]
    out.append_stdout(cmds)

app.on_ready(init)
out

Or using `asyncio`:

In [ ]:
import asyncio

app = JupyterFrontEnd()

out = Output()

async def init():
    await app.ready()
    cmds = app.commands.list_commands()[:5]
    out.append_stdout(cmds)

asyncio.create_task(init())
out

## Add your own command

Let's create a nice plot with `bqlot` and generate some random data.

See https://github.com/bqplot/bqplot/blob/master/examples/Advanced%20Plotting/Animations.ipynb for more details.

In [ ]:
import numpy as np

from bqplot import LinearScale, Lines, Bars, Axis, Figure
from ipywidgets import IntSlider

In [ ]:
xs = LinearScale()
ys1 = LinearScale()
ys2 = LinearScale()

x = np.arange(20)
y = np.cumsum(np.random.randn(20))
y1 = np.random.rand(20)

line = Lines(x=x, y=y, scales={'x': xs, 'y': ys1}, colors=['magenta'], marker='square')
bar = Bars(x=x, y=y1, scales={'x': xs, 'y': ys2}, colorpadding=0.2, colors=['steelblue'])

xax = Axis(scale=xs, label='x', grid_lines='solid')
yax1 = Axis(scale=ys1, orientation='vertical', tick_format='0.1f', label='y', grid_lines='solid')
yax2 = Axis(scale=ys2, orientation='vertical', side='right', tick_format='0.0%', label='y1', grid_lines='none')

Figure(marks=[bar, line], axes=[xax, yax1, yax2], animation_duration=1000)

We now define a function to update the data.

In [ ]:
def update_data():
    line.y = np.cumsum(np.random.randn(20))
    bar.y = np.random.rand(20)

In [ ]:
update_data()

This function will now be called when the JupyterLab command is executed.

In [ ]:
app.commands.add_command('update_data', execute=update_data, label="Update Data")

Execute it!

In [ ]:
app.commands.execute('update_data')

The slider should now be moving and taking random values.

Also the list of commands gets updated with the newly added command:

In [ ]:
assert 'random' in app.commands.list_commands()

That's great, but the command doesn't visually show up in the palette yet. So let's add it!

## Add the command to the palette

In [ ]:
from ipylab.commands import CommandPalette

In [ ]:
palette = CommandPalette()

In [ ]:
palette.add_item('update_data', 'Python Commands')

Open the command palette on the left side and the command should show now be visible.

## Remove a command

To remove a command that was previously added:

In [ ]:
app.commands.remove_command('update_data')